<a href="https://colab.research.google.com/github/ofggout880/NLP-/blob/main/MEMOIRE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from google.colab import drive
import re

# Mount Google Drive
drive.mount('/content/drive')

# Define the path to the file on Google Drive
file_path = '/content/drive/MyDrive/CODING/Python/feedbacks_SNCF.csv'

# Read the CSV file. Use encoding_errors='ignore' to skip characters
# that cannot be decoded by the default (or specified) encoding.
# Note: This means problematic characters will be missing from the data.
# A better alternative is often finding the correct 'encoding' parameter
# like encoding='latin-1' or encoding='cp1252'.
# Add the 'sep' parameter to specify the delimiter. Trying with ';'
df = pd.read_csv(file_path, encoding_errors='ignore', sep=';')

df.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,Date,campagne,Raison(s) Insatisfaction,verbatim insat,emstr
0,01/01/2025,Renouvellement CL,Je ne comprends pas pourquoi je reois cet email,"J'ai renouvel ma carte, le mieux serait donc d...",7b142afdfde15f0d567473b38219c8fc9aee887891df70...
1,01/01/2025,CDV,Je ne comprends pas pourquoi je reois cet email,"Ne pas l'envoyer, j'ai deja eu confirmation de...",21c2f8ebcc515b2d2a06cc2223170f1a8b87f6d9a386de...
2,01/01/2025,NL-VALEUR-PRO-MODEPRO-MODEPROCOMPTABLE-FR-FRFR...,Je rencontre des difficults lire cet email,je ne comprends pas ce qui m'est reproch,692192676ba9e536cca08a3f53b2d2314dad6d4b23c675...
3,01/01/2025,Alerte Petit Prix,Autre raison,Permettre FACILEMENT LA MODIFICATION du calend...,(non dfini)
4,01/01/2025,Welcome,Autre raison,"Prendre le train n'empchera l'invitable, l'ext...",(non dfini)


In [2]:
from google.colab import files
import pandas as pd
import re


# 2. Nettoyer les données (duplications, valeurs manquantes, etc.)
df.drop_duplicates(inplace=True)
df.dropna(subset=['verbatim insat'], inplace=True)  # si 'verbatim insat' est la colonne principale

# Fonction de nettoyage du texte
def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-ZÀ-ÿ0-9\\s]', '', text)
    text = re.sub(r'\\s+', ' ', text)
    return text.strip()

# 3. Application du nettoyage sur les colonnes pertinentes
df['verbatim_insat_clean'] = df['verbatim insat'].apply(clean_text)
if 'emstr' in df.columns:
    df['emstr_clean'] = df['emstr'].apply(clean_text)


In [3]:
!pip install -U spacy
!python -m spacy download fr_core_news_sm
import spacy
nlp = spacy.load("fr_core_news_sm")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 53.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [4]:
nlp = spacy.load("fr_core_news_sm")  # petit modèle français

def preprocess(text):
    doc = nlp(text)
    tokens = [token.lemma_.lower() for token in doc
              if not token.is_stop and not token.is_punct]
    return " ".join(tokens)

df['verbatim_preproc'] = df['verbatim_insat_clean'].apply(preprocess)

In [5]:
df

,Date,campagne,Raison(s) Insatisfaction,verbatim insat,emstr,verbatim_insat_clean,emstr_clean,verbatim_preproc
0,01/01/2025,Renouvellement CL,Je ne comprends pas pourquoi je reois cet email,"J'ai renouvel ma carte, le mieux serait donc d...",7b142afdfde15f0d567473b38219c8fc9aee887891df70...,jairenouvelmacartelemieuxseraitdoncdenepasmele...,7b142afdfde15f0d567473b38219c8fc9aee887891df70...,jairenouvelmacartelemieuxseraitdoncdenepasmele...
1,01/01/2025,CDV,Je ne comprends pas pourquoi je reois cet email,"Ne pas l'envoyer, j'ai deja eu confirmation de...",21c2f8ebcc515b2d2a06cc2223170f1a8b87f6d9a386de...,nepaslenvoyerjaidejaeuconfirmationdemonvoyage,21c2f8ebcc515b2d2a06cc2223170f1a8b87f6d9a386de...,nepaslenvoyerjaidejaeuconfirmationdemonvoyage
2,01/01/2025,NL-VALEUR-PRO-MODEPRO-MODEPROCOMPTABLE-FR-FRFR...,Je rencontre des difficults lire cet email,je ne comprends pas ce qui m'est reproch,692192676ba9e536cca08a3f53b2d2314dad6d4b23c675...,jenecomprendspascequimestreproch,692192676ba9e536cca08a3f53b2d2314dad6d4b23c675...,jenecomprendspascequimestreproch
3,01/01/2025,Alerte Petit Prix,Autre raison,Permettre FACILEMENT LA MODIFICATION du calend...,(non dfini),permettrefacilementlamodificationducalendrierd...,nondfini,permettrefacilementlamodificationducalendrierd...
4,01/01/2025,Welcome,Autre raison,"Prendre le train n'empchera l'invitable, l'ext...",(non dfini),prendreletrainnempcheralinvitablelextinctionav...,nondfini,prendreletrainnempcheralinvitablelextinctionav...
...,...,...,...,...,...,...,...,...
1944,10/06/2025,Post visite,Le contenu de cet email ne m'intresse pas,Intrusion dans ma vie avec les derniers voyage...,a19cf5aea351e28a772adc8b68e7b66fcee2890767a544...,intrusiondansmavieaveclesderniersvoyagespourqu...,a19cf5aea351e28a772adc8b68e7b66fcee2890767a544...,intrusiondansmavieaveclesderniersvoyagespourqu...
1945,10/06/2025,Alerte Petit Prix,Autre raison,Votre proposition ne correspond pas ce qui ta...,(non dfini),votrepropositionnecorrespondpascequitaitannonc...,nondfini,votrepropositionnecorrespondpascequitaitannonc...
1946,10/06/2025,CDV,Je ne comprends pas pourquoi je reois cet email,Deux voyages 2 mn d'intervalle ?,bf828ae7160cbf40296a6c8a7aa91f8395268b9c548a21...,deuxvoyages2mndintervalle,bf828ae7160cbf40296a6c8a7aa91f8395268b9c548a21...,deuxvoyages2mndintervall
1947,10/06/2025,CDV,Je ne suis pas satisfait(e) de mon dernier voy...,Quand les trains sont en retard aucune info ? ...,e1435f7d8295940dbe0ee2b1c3377bac4892eda6cdd476...,quandlestrainssontenretardaucuneinfolescorresp...,e1435f7d8295940dbe0ee2b1c3377bac4892eda6cdd476...,quandlestrainssontenretardaucuneinfolescorresp...
